### Importing Libraries

In [49]:
import pandas as pd
import numpy as np
import sqlite3

### Making Connection to Database

In [ ]:
conn = sqlite3.connect("my_database.db")
cursor = conn.cursor()

### Loading Dataset

In [9]:
df = pd.read_csv("datasets/batting_summary.csv").rename(columns={"Batsman_Name":"player_name"})
player_info = pd.read_csv("datasets/world_cup_players_info.csv")

In [11]:
batsman = pd.merge(player_info,df,on='player_name',how='right')

In [13]:
batsman = batsman.drop(['team_name','image_of_player','description','bowlingStyle'],axis=1)

### Removing Missing Values and Changing Datatype

In [16]:
temp = batsman.drop(batsman[batsman['Strike_Rate']=='---'].index)
temp['Strike_Rate'] = temp['Strike_Rate'].astype(float)

### Making a new Table using Group by on Batsman Name

In [19]:
def count4runs(series):
    return series.sum()*4
def count6runs(series):
    return series.sum()*6
result = temp.groupby('player_name').agg({
    'Team_Innings': 'first',    
    'Runs': 'sum',               
    'Balls': 'sum',              
    'Strike_Rate': "mean",       
    'player_name': 'size',      
    '4s':count4runs,
    '6s':count6runs,
    'Batting_Position': lambda x: round(x.mean(), 0),
    'battingStyle':'first',
    'playingRole':'first'
}).rename(columns={'player_name': 'Matches','Team_Innings':'Team'}).reset_index()

### Adding More Columns by simple calculations

In [22]:
result = result.sort_values('Matches', ascending=False)
result['Batting_Average'] = result['Runs']/result['Matches']
result['Runs By Boundaries'] = result['4s'] + result['6s']
result['Batting_Position'] = result['Batting_Position'].astype(int)
result['Boundary%'] = (result['Runs By Boundaries']*100)/result['Runs']
result['Boundary%'].fillna(0,inplace=True)

### Droping unecessary Columns

In [25]:
result = result.drop(['Runs','Balls','4s','6s','Runs By Boundaries'],axis=1)

### Rounding off Columns

In [28]:
result = result.round({'Strike_Rate':2,"Batting_Average":2,"Boundary%":2})

### Rearanging column names

In [31]:
final_batting = result.loc[:,['player_name','Team','battingStyle','playingRole','Batting_Position','Matches','Strike_Rate','Batting_Average','Boundary%']]

### Writing data into database

In [104]:
final_batting.to_sql('batters', conn, if_exists='replace', index=False)

146

In [ ]:
## Prep

### Loading Bowling Dataset

In [38]:
bowlers = pd.read_csv("datasets/bowling_summary.csv").rename(columns={'Bowler_Name':'player_name'})

In [40]:
bowlers = pd.merge(bowlers,player_info,on='player_name',how='left')

### Making a new Table using group by on Bowler Name

In [53]:
b_result = bowlers.groupby('player_name').agg({
    'team_name':'first',
    'Overs':'sum',
    'Runs':'sum',
    'Wickets':'sum',
    'Economy':'mean',
    'bowlingStyle':'first',
    'player_name':'size',
}).rename(columns={'player_name':'Matches'}).reset_index()

### Adding more Columns by simple calculations

In [56]:
b_result['Balls'] = ((b_result['Overs'].astype(int) * 6) + (b_result['Overs']-b_result['Overs'].astype(int))*10).astype(int)
b_result['strike_rate'] = b_result['Balls']/b_result['Wickets']
b_result['strike_rate'] = b_result['strike_rate'].replace(np.inf,0)
b_result['Bowling_Average'] = b_result['Runs']/b_result['Wickets']
b_result['Bowling_Average'] = b_result['Bowling_Average'].replace(np.inf,0)

### Droping unnecssary columns

In [59]:
b_temp = b_result.drop(['Overs','Balls','Runs','Wickets'],axis=1)

### Rounding off values

In [62]:
b_temp = b_temp.round({'Economy':2,"strike_rate":2,"Bowling_Average":2})

### Changing order of columns

In [65]:
final_bowling = b_temp.loc[:,['player_name','team_name','bowlingStyle','Matches','Economy','strike_rate','Bowling_Average']]

### Extracting Top Fast Bowlers

In [109]:
final_bowling.to_sql('bowlers', conn, if_exists='replace', index=False)

102

## Preparing All Rounders Dataset

### Merging Bowling and Batting Data

In [67]:
all_rounder = pd.merge(final_bowling,final_batting,on='player_name',how='inner')

### Droping unnecessary columns

In [70]:
all_rounder = all_rounder.drop(['team_name','bowlingStyle','Bowling_Average','battingStyle','Boundary%'],axis=1)

### Changing Columns name

In [73]:
all_rounder = all_rounder.rename(columns = {'strike_rate':'Bowling_Sr','Matches_y':'Innings_batted','Matches_x':'Innings_bowled','Strike_Rate':'Batting_Sr'})

### Rearranging Columns

In [76]:
all_rounder = all_rounder.loc[:,['player_name','Team','playingRole','Innings_batted','Batting_Position','Batting_Average','Batting_Sr','Innings_bowled','Economy','Bowling_Sr']]

### Writing data into database

In [102]:
all_rounder.to_sql('all_rounders', conn, if_exists='replace', index= False)

97

102

97